In [ ]:
import secrets
import hashlib

##
## Enjoy Ilyes!!!!
##

#
# Local Simulation of CGGMP21 Threshold ECDSA Signing Protocol
#
# This script simulates the cryptographic flow of the CGGMP21 protocol for
# distributed key generation (DKG) and threshold signing.
#
# DISCLAIMER: This is a simplified simulation for educational purposes.
# It does NOT include crucial security components of the real protocol, such as:
#   - Zero-Knowledge Proofs (for proving correctness without revealing secrets)
#   - Secure point-to-point communication channels
#   - Robust handling of malicious adversaries (it assumes all parties are "honest-but-curious")
#
# The goal is to demonstrate the core mathematical concepts.
#

# --- 1. Curve Parameters (secp256k1) & Helper Functions ---
# We use the same battle-tested curve and math functions.

P = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
A = 0
B = 7
Gx = 0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798
Gy = 0x483ADA7726A3C4655DA4FBFC0E1108A8FD17B448A68554199C47D08FFB10D4B8
G = (Gx, Gy)
N = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141

def inv(n, prime=N):
    return pow(n, prime - 2, prime)

def point_add(p1, p2):
    if p1 is None: return p2
    if p2 is None: return p1
    x1, y1 = p1
    x2, y2 = p2
    if x1 == x2 and y1 != y2: return None
    if x1 == x2:
        m = (3 * x1 * x1 + A) * inv(2 * y1, P)
    else:
        m = (y2 - y1) * inv(x2 - x1, P)
    x3 = (m * m - x1 - x2) % P
    y3 = (m * (x1 - x3) - y1) % P
    return (x3, y3)

def scalar_mult(k, point=G):
    if k % N == 0 or point is None: return None
    result = None
    addend = point
    while k:
        if k & 1: result = point_add(result, addend)
        addend = point_add(addend, addend)
        k >>= 1
    return result

# --- 2. Lagrange Coefficient Calculation ---
def compute_lagrange_coeff(party_ids, target_id):
    """
    Computes the Lagrange coefficient lambda_i for a given party i at x=0.
    lambda_i = product( j / (j - i) for j in S if j != i), where S is the set of party IDs.
    """
    numerator = 1
    denominator = 1
    for j in party_ids:
        if j != target_id:
            numerator = (numerator * j) % N
            denominator = (denominator * (j - target_id)) % N
    return (numerator * inv(denominator, N)) % N

# --- 3. Signer Class Definition ---
class Signer:
    """Represents a single participant in the DKG and signing protocol."""
    def __init__(self, party_id, num_parties, threshold):
        self.id = party_id
        self.num_parties = num_parties
        self.threshold = threshold

        # DKG state
        self.poly_coeffs = None
        self.commitments = None
        self.received_shares = {}
        self.private_key_share = None # This is x_i = f(i)
        self.public_key_share = None # This is Y_i = x_i * G

        # Signing state
        self.k_share = None # k_i
        self.R_share = None # R_i = k_i * G

    def dkg_round_1_create_poly_and_commitments(self):
        """Each party creates its own secret polynomial and commitment."""
        # Create a polynomial of degree t (threshold - 1)
        # f_i(z) = a_i0 + a_i1*z + ... + a_it*z^t
        self.poly_coeffs = [secrets.randbelow(N) for _ in range(self.threshold)]
        # The commitment is C_i = [a_i0*G, a_i1*G, ..., a_it*G]
        self.commitments = [scalar_mult(c) for c in self.poly_coeffs]

    def dkg_round_2_get_share_for_party(self, party_j_id):
        """Calculates the share s_ij = f_i(j) for party j."""
        y = 0
        # Evaluate polynomial f_i at point x=party_j_id
        for coeff in reversed(self.poly_coeffs):
            y = (y * party_j_id + coeff) % N
        return y

    def dkg_round_3_verify_and_store_share(self, from_party_id, share, commitments):
        """Verify the received share s_ji against party j's commitment C_j."""
        # check that s_ji * G == sum( (i^k * C_jk) for k in 0..t )
        lhs = scalar_mult(share)
        rhs = None
        for k in range(self.threshold):
            # C_jk is commitments[k] for party j (from_party_id)
            # We evaluate at our own id, i.
            term = scalar_mult(pow(self.id, k, N), commitments[k])
            rhs = point_add(rhs, term)

        if lhs != rhs:
            raise ValueError(f"Party {self.id} received a bad share from Party {from_party_id}")
        self.received_shares[from_party_id] = share

    def dkg_round_4_compute_key_shares(self):
        """Compute final private key share x_i = sum(s_ji for j in 1..n)."""
        self.private_key_share = sum(self.received_shares.values()) % N
        self.public_key_share = scalar_mult(self.private_key_share)

    def sign_round_1_create_nonce_share(self):
        """Each signing party generates a secret nonce share k_i and a public R_i."""
        self.k_share = secrets.randbelow(N)
        self.R_share = scalar_mult(self.k_share)
        return self.R_share

# --- 4. ECDSA Verification Function ---
def ecdsa_verify(public_key, msg_hash, signature):
    r, s = signature
    if not (1 <= r < N and 1 <= s < N):
        return False

    z = int.from_bytes(msg_hash, 'big')
    s_inv = inv(s, N)
    u1 = (z * s_inv) % N
    u2 = (r * s_inv) % N

    p1 = scalar_mult(u1, G)
    p2 = scalar_mult(u2, public_key)
    R_check = point_add(p1, p2)

    if R_check is None:
        return False

    return R_check[0] % N == r

# --- Main Simulation ---
if __name__ == '__main__':
    N_PARTIES = 5
    THRESHOLD = 3 # This means t=2 in polynomial terms, requires 3 parties to sign.

    print(f"--- CGGMP21 Simulation ---")
    print(f"Total Parties (n): {N_PARTIES}, Threshold (t+1): {THRESHOLD}\n")

    # === 1. Setup ===
    parties = [Signer(i, N_PARTIES, THRESHOLD) for i in range(1, N_PARTIES + 1)]

    # === 2. Distributed Key Generation (DKG) ===
    print("--- Phase 1: Distributed Key Generation ---")

    # Round 1: Each party creates a polynomial and broadcasts commitments
    all_commitments = {}
    for p in parties:
        p.dkg_round_1_create_poly_and_commitments()
        all_commitments[p.id] = p.commitments
        print(f"Party {p.id} created poly and commitments.")

    # Round 2 & 3: Parties exchange and verify shares
    for sender in parties:
        for receiver in parties:
            if sender.id != receiver.id:
                share = sender.dkg_round_2_get_share_for_party(receiver.id)
                receiver.dkg_round_3_verify_and_store_share(sender.id, share, all_commitments[sender.id])
    print("All parties exchanged and verified shares successfully.")

    # Round 4: Each party computes its final key share
    for p in parties:
        self_share = p.dkg_round_2_get_share_for_party(p.id)
        p.received_shares[p.id] = self_share
        p.dkg_round_4_compute_key_shares()
    print("All parties computed their final private key shares (x_i).")

    # Aggregate the public key Y = sum(Y_i0) where Y_i0 is commitment to secret f_i(0)
    agg_public_key = None
    for p in parties:
        public_share = p.commitments[0]
        agg_public_key = point_add(agg_public_key, public_share)

    print("\nDKG Complete!")
    print(f"Aggregated Public Key (Y):\n  x: {hex(agg_public_key[0])}\n  y: {hex(agg_public_key[1])}\n")


    # === 3. Threshold Signing ===
    print("--- Phase 2: Threshold Signing ---")
    message = "CGGMP21 is a cool protocol"
    msg_hash = hashlib.sha256(message.encode('utf-8')).digest()

    # Select a subset of THRESHOLD signers
    signing_parties_ids = list(range(1, THRESHOLD + 1))
    signing_parties = [p for p in parties if p.id in signing_parties_ids]
    print(f"Message: '{message}'")
    print(f"Signing with parties: {signing_parties_ids}\n")

    # Signing Round 1: Create nonce shares and aggregate R
    R_shares = {}
    for sp in signing_parties:
        R_shares[sp.id] = sp.sign_round_1_create_nonce_share()

    R_agg = None
    for R_i in R_shares.values():
        R_agg = point_add(R_agg, R_i)

    r = R_agg[0] % N
    if r == 0:
        raise RuntimeError("r is zero, must restart signing round")

    print(f"Aggregated R computed. Signature 'r' value is: {hex(r)}")

    # Signing Round 2: Compute final signature 's'
    # The secret key x is reconstructed implicitly using Lagrange interpolation.
    # x = sum(lambda_i * x_i) where x_i is the private key share of party i.
    m_int = int.from_bytes(msg_hash, 'big')

    # Implicitly reconstruct the full private key 'x' using Lagrange coefficients
    x_reconstructed = 0
    for sp in signing_parties:
        lagrange_coeff = compute_lagrange_coeff(signing_parties_ids, sp.id)
        x_reconstructed = (x_reconstructed + lagrange_coeff * sp.private_key_share) % N

    # The aggregate nonce 'k' is the sum of the nonce shares
    k_agg = sum(sp.k_share for sp in signing_parties) % N

    # Calculate signature s = k^-1 * (m + r*x)
    s = (inv(k_agg) * (m_int + r * x_reconstructed)) % N

    if s == 0:
        raise RuntimeError("s is zero, must restart signing round")

    final_signature = (r, s)
    print(f"Final signature 's' value is: {hex(s)}\n")

    # === 4. Verification ===
    print("--- Phase 3: Verification ---")
    is_valid = ecdsa_verify(agg_public_key, msg_hash, final_signature)
    print(f"Signature valid: {is_valid}")

    if is_valid:
        print("\nSUCCESS: The threshold signature was verified correctly with the aggregated public key.")
    else:
        print("\nFAILURE: The signature verification failed.")


--- CGGMP21 Simulation ---
Total Parties (n): 5, Threshold (t+1): 3

--- Phase 1: Distributed Key Generation ---
Party 1 created poly and commitments.
Party 2 created poly and commitments.
Party 3 created poly and commitments.
Party 4 created poly and commitments.
Party 5 created poly and commitments.
All parties exchanged and verified shares successfully.
All parties computed their final private key shares (x_i).

DKG Complete!
Aggregated Public Key (Y):
  x: 0xeefa616da9ca392fac32d775a7839d0fad6fce344c754bd0efddd13de19040a7
  y: 0x98d40a49206548109d69b4f6b10650314445404e4ab853ce1f1236c2e3e18d15

--- Phase 2: Threshold Signing ---
Message: 'CGGMP21 is a cool protocol'
Signing with parties: [1, 2, 3]

Aggregated R computed. Signature 'r' value is: 0x4946041c86e41889ef7e70337fe20f778ef7ca2fd6d13c3089b5ab9ed060c4f2
Final signature 's' value is: 0xba80e8878c6cfb26edb6e1c26e75c6e39bc661ad1f89f7e41858b07de3bcfb3b

--- Phase 3: Verification ---
Signature valid: True

SUCCESS: The threshold s